## Import Libraries

In [ ]:
# Import numerical tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns              #Similar to Matplotlib but it's more simple to manage dataframe

# Import ASE tools
from ase.calculators.vasp import Vasp
from ase.db import connect
from ase.visualize import view
from ase.io import write, read

# Import OS and Subprocess libraries
import os
import subprocess

# Use OS and Subprocess to explore folders

In [ ]:
# Using os.system you can execute command batch but you can not storage in a variable
batch_command = "ls -d */"

print("Results using os.system")
a = os.system(batch_command)
# print("Result if we try to storage the variable ", a) # It is a batch result, 0 if the command run correct and 1 if it not




# Using subprocess you can do the same but you can define a variable and storage the result
result_batch_command = subprocess.check_output(batch_command, shell = True)
result_batch_command = result_batch_command.decode('utf-8') # Convert a bytes variable to string variable
result_batch_command = result_batch_command.splitlines()    # Split by /n

print("")
print("Results using subprocess")
for res_txt in result_batch_command:
    print(res_txt)

# Use ASE to read VASP results

In [ ]:
# If you have your DFT results from VASP you can use ASE to verify some of your results

# Using Vasp Calculators from ASE you can read the correspoding folder
Directory = result_batch_command[0]   # result_batch_command[0] should be a directory with the final VASP output files  
print("Directory read: ", Directory, "\n")
calc = Vasp(directory = Directory)
calc.read()

# Get the atoms final position
atoms = calc.get_atoms()
# view(atoms)

# Some validation
print('Converged? ', calc.converged, "\n")

print("Energy when sigma -> 0, ", atoms.get_potential_energy(), " eV")
# You can obtain this value using the terminal:
pot_energy = subprocess.check_output("grep sigma " + Directory + "OUTCAR | tail -1", shell = True)
print("Energy Line using the batch")
print(pot_energy,"\n")

print("Forces per atom in eV/Å")
print(atoms.get_forces())

## Look up Convergence Criteria

In [ ]:
# Define the default EDIFF and EDIFFG values
EDIFF_value  = 1e-4
EDIFFG_value = EDIFF_value*10

# Read INCAR File to look up for the correspoding tag
f = open(Directory + 'INCAR')
alllines=f.readlines()
f.close()
n = len(alllines)

for i in reversed(range(n)):
    line = alllines[i]
    
    # Get EDIFF Value
    if 'EDIFF' in line:
        EDIFF_value = float(line[9:])
    
    # Get EDIFFG Value
    if 'EDIFFG' in line:
        EDIFFG_value = float(line[9:])

    
print("EDIFF value, " , EDIFF_value, "eV")
if EDIFFG_value >= 0:
    print("EDIFFG Positive, the relaxation is stopped when the change of the total energy is smaller than EDIFFG between two ionic steps, ", EDIFFG_value)
else:
    print("EDIFFG Negative, the relaxation is stopped when the norms of all the forces are smaller than |EDIFFG|, ", EDIFFG_value)
    


In [ ]:
try:
    check_validation_forces = subprocess.check_output("grep 'FORCES:' "+ Directory + "OUTCAR", shell=True)
except:
    check_validation_forces = -1
    
if check_validation_forces != -1:
    f = os.popen("grep 'FORCES:' "+ Directory +  "OUTCAR")
    fmax = []
    for line in f:
        fields = line.split()
        fmax.append(float(fields[4]))
    fmax_value = fmax[-1]
    print('Max force is {0} eV/A'.format(fmax_value))
    
else:
    f = open(Directory + 'OUTCAR')
    alllines=f.readlines()
    f.close()
    n = len(alllines)
    for i in reversed(range(n)):
        line = alllines[i]
        if 'TOTAL-FORCE' in line:
            break
    i+=2
    forces = []
    for atom in atoms:
        line = alllines[i]
        values = line.split()
        xforce = abs(float(values[3]))
        yforce = abs(float(values[4]))
        zforce = abs(float(values[5]))
        force = np.linalg.norm([xforce,yforce,zforce])
        forces.append(force)
        i+=1
    fmax = round(max(forces),5)
    fmax_value = fmax
    print('Max force is {0} eV/A'.format(fmax))

In [ ]:
n_iteration = subprocess.check_output("grep 'F=' "+ Directory + "OSZICAR | tail -1", shell=True)
n_iteration = n_iteration.decode('utf-8') # Convert a bytes variable to string variable
n_iteration = int(n_iteration.split()[0])

# print("Number of Iterations, ", n_iteration)

f = open(Directory + 'OSZICAR')
alllines=f.readlines()
f.close()
n = len(alllines)

it = 0

for i in reversed(range(n)):
    line = alllines[i]
    if 'F=' in line:
        if it == 0:
            index_last_iteration = i
        elif it == 1:
            index_last_iteration_aux = i
            
        if it == 1:
            break
            
        it+=1
        
DiffEnergy_Iter = []

for i in range(index_last_iteration_aux+2, index_last_iteration):
    line = alllines[i].split()
    DiffEnergy_Iter.append(float(line[3]))

Change_TotalEnergy = DiffEnergy_Iter[-1]
print("Change of Total Energy, ",Change_TotalEnergy, "eV")

In [ ]:
if EDIFFG_value >= 0:
    if EDIFFG_value < abs(Change_TotalEnergy):
        print("Positive EDIFFG, Achieve ionic relaxation criteria?, False")
    else:
        print("Positive EDIFFG, Achieve ionic relaxation criteria?, True")
else:
    if abs(EDIFFG_value) < fmax_value:
        print("Negative EDIFFG, Achieve ionic relaxation criteria?, False")
    else:
        print("Negative EDIFFG, Achieve ionic relaxation criteria?, True")

# Read Folders with DFT Results and Generate a Database

## Read VASP Output and Write in Database

In [ ]:
DirectoryName = result_batch_command[0]

# Read Folder with VASP Output
calc   = Vasp(directory = DirectoryName)
calc.read()

## Connect to Database or Generate Database

In [ ]:
# We need to establish a connection to a database
DatabaseName = 'Database_Default.db'

# Use append=False to start a new database.
con = connect(DatabaseName, append=False)

# Write in the Database
con.write(calc.get_atoms())

## Get Name of Default Keys

In [ ]:
db = connect(DatabaseName)
row = db.get(id = 1)

for key in row:
    print('{0}'.format(key))  

## Get Values of those defaults keys

In [ ]:
for key in row:
    print('{0:22}: {1}'.format(key, row[key]))  

# You can add Custom Keys to your database

In [ ]:
# We need to establish a connection to a database
DatabaseName = 'Database_Custom.db'

# Use append=False to start a new database.
con = connect(DatabaseName, append=False)

Directory = result_batch_command[0]

# Read Folder with VASP Output
calc      = Vasp(directory = Directory)
calc.read()

# Write in the Database
con.write(calc.get_atoms(), Functional = "PBE", VaspVersion = "5.4.1")

In [ ]:
db = connect(DatabaseName)
row = db.get(id = 1)

for key in row:
    print('{0:22}: {1}'.format(key, row[key]))  

## Recover CONTCAR from ASE Database

In [ ]:
# We need to establish a connection to a database
DatabaseName = 'Database_Custom_Expand.db'

# Use append=False to start a new database.
con = connect(DatabaseName, append=False)

for i in result_batch_command:
    Directory = i

    # Read Folder with VASP Output
    calc      = Vasp(directory = Directory)
    calc.read()

    # Write in the Database
    con.write(calc.get_atoms())

In [ ]:
# Connect to database
db_connected = connect(DatabaseName)

i = 0

# Iteration per data (each row is a structure)
for row in db_connected.select():
    atoms = row.toatoms()
    
    # Used to create the directory in the case that it does not exist
    boolean_folder = os.path.isdir("CONTCAR_files")
    if boolean_folder == False:
        os.mkdir("CONTCAR_files")
    
    # Write the CONTCAR file
    atoms.write("CONTCAR_files/CONTCAR_{i}.vasp".format(i=i))
    
    i += 1
        

## Using ASE Database to Generate Dataframe

In [ ]:
Dataframe_from_ASEDatabase = pd.DataFrame(columns=['ID', 'Calculator', 'CalculatorVersion','PotentialEnergy'])


db_Z2H2_PBE = connect('Database_Custom.db')


for row in db_Z2H2_PBE.select():
    atoms                  = row.toatoms()
    EnergyCurrent          = atoms.get_potential_energy()
    
    id                     = row.id
    calculator             = row.calculator
    calculatorVersion        = row.VaspVersion
    
    auxArray = np.array([id, calculator, calculatorVersion, EnergyCurrent])
        
    Dataframe_from_ASEDatabase.loc[len(Dataframe_from_ASEDatabase.index)] = auxArray


## Convert to Numeric the corresponding Features
cols = Dataframe_from_ASEDatabase.columns.drop(['Calculator', 'CalculatorVersion'])
Dataframe_from_ASEDatabase[cols] = Dataframe_from_ASEDatabase[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
## Get the first 5 item in the dataframe
Dataframe_from_ASEDatabase.head()